In [1]:
import numpy as np
from scipy.optimize import minimize
from formate_matrix_toMLData import *
from data_generator import *


In [65]:
#データの読み込み
file_path = "data/non_fix_test/var_setSize5000-10000/aa7309_4.csv"
all = np.loadtxt(file_path, delimiter=",")
mt = matrix_trimer(all)
trm = mt.trim_transitionRateMatrix()
data = mt.trim_data()
fm = formate_dataMatrix()
vec = fm.GetOutputVector_byUpperTriangle(trm)
rand_vec = np.random.uniform(0,1,6)

In [66]:
print(rand_vec)

[0.57469355 0.32645837 0.28173407 0.17081867 0.30934934 0.45580292]


In [67]:
#パラメタベクトルから推移率行列の形にするメソッド
def vec_to_UpperTriMatrix(vector):
    count = 0
    i = 1
    while count < len(vector):
        count += i
        i += 1
    if(count != len(vector)):
        raise IndexError("vec length is incorrect!")
    matrix = np.zeros((i,i))
    iterator =iter(vector)
    for j in range(0,i):
        for k in range(j+1, i):
            matrix[j][k] = max(0,next(iterator))
    for j in range(0,j):
        sum = np.sum(matrix[j,:])
        matrix[j][j] = -sum
    return matrix
    
#尤度関数の定義
def f(x):
    """_summary_

    Args:
        x : vector
    """
    sum = 0
    Q = vec_to_UpperTriMatrix(x)
    for sample in(data):
        pm = CalcProbmatrix(Q,sample[2]).calcProbmatrix()
        sum -= np.log(pm[int(sample[0])-1][int(sample[1])-1]+1e-6)
    return sum
        
        

In [68]:
a = vec_to_UpperTriMatrix(vec)
print(a)

[[-0.61571671  0.1820844   0.13636713  0.29726518]
 [ 0.         -0.49634038  0.16617261  0.33016777]
 [ 0.          0.         -0.35669097  0.35669097]
 [ 0.          0.          0.          0.        ]]


In [69]:
result = minimize(f, vec,method='BFGS', options={'gtol': 1e-5, 'maxiter': 10000, 'disp': True})

         Current function value: 4533.019918
         Iterations: 16
         Function evaluations: 691
         Gradient evaluations: 97


/opt/anaconda3/envs/kisozemi/lib/python3.11/site-packages/scipy/optimize/_minimize.py:708: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


In [70]:
print("最大値のx",result.x)

最大値のx [0.1567208  0.15250863 0.31513548 0.17289093 0.34543528 0.35281609]
